In [1]:
import json
import math
import numpy as np
import random
import scipy.stats

In [2]:
from utils import join_reference_set_and_results, compute_sign_rate_found, create_map

In [3]:
with open('observational_results.txt') as f:
    observational_results = [json.loads(a) for a in f]

reference_set_names = ['../raw_reference_set.txt', '../reference_set.txt', '../multi_rct_reference_set.txt']
reference_sets = []

for reference_set_name in reference_set_names:
    with open(reference_set_name) as f:
        reference_sets.append([json.loads(a) for a in f])

In [4]:
filtered_maps = []
for reference_set in reference_sets:
    joined = join_reference_set_and_results(reference_set, observational_results)
    filtered_maps.append(create_map([a for a in joined if 'postmean' in a]))

In [5]:
total_keys = set()
for filtered_map in filtered_maps:
    total_keys |= set(filtered_map.keys())

total_keys = sorted(list(total_keys))

In [6]:
random.seed(123154)

performances = [[] for _ in range(len(filtered_maps))]

method_type = 'cox'
method_key = 'logistic_match'

lower_limit = 1.5
upper_limit = 4

def get_performance_at(sample, filtered_map, value):
    filtered = [filtered_map.get(k) for k in sample if k in filtered_map]
    sorted_by_bayes = sorted(filtered, key=lambda v: v['postmean'])
    
    postmean, fraction_correct, fraction_found, num_significant = compute_sign_rate_found(sorted_by_bayes, method_type, method_key)

    return np.interp(-value, -postmean, fraction_correct), np.interp(-value, -postmean, fraction_found)


def get_average_performance_between(sample, filtered_map, lower_limit, upper_limit):
    filtered = [filtered_map.get(k) for k in sample if k in filtered_map]
    sorted_by_bayes = sorted(filtered, key=lambda v: v['postmean'])
    
    postmean, fraction_correct, fraction_found, num_significant = compute_sign_rate_found(sorted_by_bayes, method_type, method_key)

    first_less_than_limit = np.argmax(postmean < upper_limit)
    assert postmean[first_less_than_limit] < upper_limit
    assert postmean[first_less_than_limit - 1] >= upper_limit
    
    postmean[first_less_than_limit - 1] = upper_limit
    
    postmean = postmean[first_less_than_limit - 1:]
    fraction_correct = fraction_correct[first_less_than_limit - 1:]
    fraction_found = fraction_found[first_less_than_limit - 1:]

    first_greater_than = np.argmax(postmean <= lower_limit)
    assert postmean[first_greater_than] <= lower_limit
    assert postmean[first_greater_than - 1] > lower_limit
    
    postmean[first_greater_than - 1] = lower_limit
    
    postmean = postmean[:first_greater_than]
    fraction_correct = fraction_correct[:first_greater_than]
    fraction_found = fraction_found[:first_greater_than]

    return -np.trapz(fraction_correct, postmean) / (upper_limit - lower_limit), -np.trapz(fraction_found, postmean) / (upper_limit - lower_limit)

print(lower_limit, upper_limit)
for reference_set_name, filtered_map in zip(reference_set_names, filtered_maps):
    print(reference_set_name, get_performance_at(total_keys, filtered_map, 2.5))

for i in range(1000):
    if i % 100 == 0:
        print(i)
    sample = random.choices(total_keys, k=len(total_keys))
    for performance, filtered_map in zip(performances, filtered_maps):
        performance.append(get_performance_at(sample, filtered_map, 2.5))

1.5 4
../raw_reference_set.txt (0.8480380935141079, 0.40602321589902685)
../reference_set.txt (0.9640821238961352, 0.5177798422462125)
../multi_rct_reference_set.txt (0.8478283725884798, 0.5442462158317966)
0
100
200
300
400
500
600
700
800
900


In [8]:
for i in range(1, len(reference_set_names)):
    print(reference_set_names[0], reference_set_names[i])
    for index, name in enumerate(["concordance", "recovery"]):
        helper = lambda a: a[index]
        print(name)

        for other in (0, i):
            print(reference_set_names[other], f'{100 * helper(get_performance_at(total_keys, filtered_maps[other], 2)):0.2f}')
        
        deltas = []
        for a, b in zip(performances[0], performances[i]):
            deltas.append(helper(a) - helper(b))
    
        z = np.mean(deltas) / np.std(deltas, ddof=1)
        p = scipy.stats.norm.sf(abs(z)) * 2
        print(z, p)
    
        print(np.quantile(deltas, [0.025, 0.975]))
        
        print(np.mean(deltas), np.std(deltas,ddof=1))

../raw_reference_set.txt ../reference_set.txt
concordance
../raw_reference_set.txt 71.84
../reference_set.txt 74.58
-2.192979094129564 0.028308885362442898
[-0.20483625 -0.00974334]
-0.10770897371983018 0.049115367313878555
recovery
../raw_reference_set.txt 31.70
../reference_set.txt 40.82
-1.9413986626508721 0.05220994039545384
[-0.22109196 -0.00110216]
-0.1115835688837142 0.05747586574070934
../raw_reference_set.txt ../multi_rct_reference_set.txt
concordance
../raw_reference_set.txt 71.84
../multi_rct_reference_set.txt 68.36
0.09312430377062021 0.9258048097133519
[-0.12134034  0.16246384]
0.0067721553102932045 0.07272167453701546
recovery
../raw_reference_set.txt 31.70
../multi_rct_reference_set.txt 38.29
-1.678800991419248 0.09319083533721673
[-0.2862471   0.03311037]
-0.13476404184488627 0.08027398276132633
